In [ ]:
import pdfplumber 
import pandas as pd  
import 


TypeError: 'NoneType' object is not iterable

In [5]:
rows = []
with pdfplumber.open("opal_activity.pdf") as pdf:
    for page in pdf.pages:
        tbl = page.extract_table()          # or page.extract_tables()[0]  
        if tbl:                             # <- skip pages without tables
            rows.extend(tbl)

df = pd.DataFrame(rows[1:], columns=rows[0])
df.to_csv("opal_trips.csv", index=False)

IndexError: list index out of range